In [14]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import os

In [15]:
def dist(v1,v2):
    return np.sqrt(np.sum((v1-v2)**2))


In [16]:
def knn(X,Y,query_point,k=5):
    m=X.shape[0]             # n.o of points in training dataset 
    distances=[]             # calculate distance of all points with query point
    for i in range(m):           # for each point
        d=dist(query_point,X[i])
        distances.append((d,Y[i]))      # appending label too to be able to tell which distance is for dog and which is for cat
    
    distances=sorted(distances)[:k]      # take first k elements  , writing nothing onleft of semicolon means its 0 by default
    distances=np.array(distances)        # convert list to array so that we can apply np operations
    labels=distances[:,1]                #  select all rows and second colmn
    labels,freq=np.unique(labels,return_counts=True)    # return 2 arrays see below example
    pred=labels[np.argmax(freq)]                        # take the highest freq class 
    return int(pred)

In [17]:
files=os.listdir()
d={}
cnt=0
X=[]
Y=[]

for f in files:
    if f.endswith(".npy"):
        data=np.load(f)
        print(data.shape,f)
        labels=np.ones((data.shape[0],1))*cnt;
        
        X.append(data)
        Y.append(labels)
        d[cnt]=f[:-4]    # slice the string , i.e the file name
        cnt+=1
        

(20, 100, 100, 3) jai.npy


In [18]:
X = np.vstack(X)
Y = np.vstack(Y)
Y = Y.reshape((20,))
print(X.shape,Y.shape)

(20, 100, 100, 3) (20,)


In [19]:
cam=cv2.VideoCapture(0)                  # camera object captured , later it will have to be released 
detector=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

while True:                         # so that we will break from loop oce we cut the webcam window
    ret,frame=cam.read()
    
    if ret==False:                  # ret is what it receives from webcam
        continue
        
    faces=detector.detectMultiScale(frame,1.3,5)
    
    if(len(faces)==0):
        continue               # no face detected
    
    for face in faces:
        x,y,w,h=face
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)    #frame= removed
        cropped_face=frame[y:y+h,x:x+w]
        cropped_face=cv2.resize(cropped_face,(100,100))
        #Find out the name of the face
        pred=knn(X,Y,cropped_face,k=5)
        pred_name=d[pred]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1.0,(255,255,255),lineType=cv2.LINE_AA)
        
    cv2.imshow("frame",frame)
    
    key_pressed=cv2.waitKey(1) & 0xFF
    
    if(key_pressed==ord('e')):
        break
                                  # to explicitly break from the loop

cam.release()                   # release webcam
cv2.destroyAllWindows()  